In [374]:
import os
import shutil  # To move files from tmp to the final folder
from typing import Any
from pydantic import BaseModel
import time
from PyPDF2 import PdfFileReader, PdfFileWriter
from unstructured.partition.pdf import partition_pdf

class Element(BaseModel):
    type: str
    text: Any
    page_number: int
    table_as_html: Any
    image_path: str
    coordinate : Any

In [375]:
import pickle

# Define the path to your pickle file
# pickle_file_path = "/home/jupyter/project/VP_storage/output_books/gastrointestinal-imaging-the-requisites-fourth-edition.pkl" #done
# pickle_file_path = "/home/jupyter/project/VP_storage/output_books/Radiology Illustrated_ Hepatobiliary and Pancreatic Radiology.pkl" #done
# pickle_file_path = "/home/jupyter/project/VP_storage/output_books/Liver imaging _ MRI with CT correlation.pkl" #done
pickle_file_path = "/home/jupyter/project/VP_storage/output_books/Diagnostic Imaging_ Gastrointestinal.pkl" #error happened -- skip
#pickle_file_path = "/home/jupyter/project/VP_storage/output_books/CT and MRI of the Whole Body, 2-Volume Set, 6e, Volume I.pkl" #error happened -- skip
#pickle_file_path = "/home/jupyter/project/VP_storage/output_books/Mayo Clinic Gastrointestinal Imaging Review.pkl" #(dont use the same rule for this, as it has many comparisons and intended to be close to each other)
# Open the file in binary read mode and load the object

# pickle_file_path = "/home/jupyter/project/VP_storage/output_papers/an-et-al-2020-abbreviated-mri-for-hepatocellular-carcinoma-screening-and-surveillance.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_papers/fujita-et-al-2019-hyperintense-liver-masses-at-hepatobiliary-phase-gadoxetic-acid-enhanced-mri-imaging-appearances-and.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_papers/m-cunha-et-al-2021-how-to-use-li-rads-to-report-liver-ct-and-mri-observations.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_papers/RoleofMRIinEvaluationofSpectrumofLiverLesionsinCirrhoticPatients-JAPI.pkl" 
#pickle_file_path = "/home/jupyter/project/VP_storage/output_papers/WJH-13-1936.pkl" 

# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/Diagnosis _ RADS - Reporting and Data Systems Support.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/Diagnostic Categories _ RADS - Reporting and Data Systems Support.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/Getting Started _ RADS - Reporting and Data Systems Support.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/Imaging Features _ RADS - Reporting and Data Systems Support.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/LI-RADS 2018 Core.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/LI-RADS CTMR Nonradiation TRA v2024 Core.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/LI-RADS CTMR Radiation TRA v2024 Core.pkl" 


# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/LIRADS Lexicon Table.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/Management _ RADS - Reporting and Data Systems Support.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/Reporting _ RADS - Reporting and Data Systems Support.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/Treatment Response _ RADS - Reporting and Data Systems Support.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/What's New in v2018 _ RADS - Reporting and Data Systems Support.pkl" 
# pickle_file_path = "/home/jupyter/project/VP_storage/output_system_guides/LI-RADS US Surveillance v2024 Core.pkl" 


with open(pickle_file_path, 'rb') as file:
    image_elements, captions, raw_elements,a,b= pickle.load(file)

In [376]:
import os
path = "/home/jupyter/project/VP_storage/output_books"
#path = "/home/jupyter/project/VP_storage/output_papers"
#path = "/home/jupyter/project/VP_storage/output_system_guides"
os.chdir(path)

In [377]:
import pandas as pd
image_elements_flat = [
    {
        'Element_Type': element.type,
        'Text': element.text,
        'Page_Number': element.page_number,
        'Image_Path': element.image_path,
        'Coordinates': element.coordinate
    }
    for element in image_elements
]

# Convert the flattened structure to a DataFrame
df_flattened = pd.DataFrame(image_elements_flat)

In [378]:
df_flattened['coordinate_bottom_left'] = df_flattened['Coordinates'].apply(lambda x:x[1])

In [379]:
from math import sqrt
from itertools import combinations

In [380]:
df = df_flattened

In [381]:
import os
import itertools
from PIL import Image
import imagehash
import pandas as pd
from math import sqrt
import cv2
from scipy.special import rel_entr


# Function to calculate the midpoint of an image based on four coordinates
def calculate_midpoint(coordinates):
    x_coords = [coord[0] for coord in coordinates]
    y_coords = [coord[1] for coord in coordinates]
    midpoint = (sum(x_coords) / 4, sum(y_coords) / 4)
    return midpoint

# Function to calculate distance between two midpoints
def calculate_distance(point1, point2):
    return sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Function to calculate image hash difference
def calculate_hash_difference(image_path1, image_path2):
    # Calculate the average hash for both images
    hash1 = imagehash.average_hash(Image.open(image_path1))
    hash2 = imagehash.average_hash(Image.open(image_path2))
    
    # Calculate the difference between the two hashes
    return hash1 - hash2

# Function to get the image size
def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size  # Returns (width, height)

import cv2
import numpy as np


# def calculate_orb_similarity(image_path1, image_path2):
#     # Load the images
#     image1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
#     image2 = cv2.imread(image_path2, cv2.IMREAD_GRAYSCALE)

#     # Check if images were successfully loaded
#     if image1 is None or image2 is None:
#         print("Error loading one of the images.")
#         return 0

#     # Initiate ORB detector
#     orb = cv2.ORB_create()

#     # Find keypoints and descriptors with ORB
#     keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
#     keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

#     # If no descriptors found in either image, return 0 matches
#     if descriptors1 is None or descriptors2 is None:
#         return 0

#     # Convert descriptors to CV_8U if they are not already (required by NORM_HAMMING)
#     if descriptors1.dtype != np.uint8:
#         descriptors1 = np.uint8(descriptors1)
#     if descriptors2.dtype != np.uint8:
#         descriptors2 = np.uint8(descriptors2)

#     # Use BFMatcher to find the matches
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
#     matches = bf.match(descriptors1, descriptors2)

#     # Sort matches based on the distance between descriptors
#     matches = sorted(matches, key=lambda x: x.distance)

#     # Filter matches based on a distance threshold
#     good_matches = [m for m in matches if m.distance < 30]

#     # Return the ratio of good matches to the average number of keypoints
#     return len(good_matches) / ((len(keypoints1) + len(keypoints2)) / 2)


# Function to calculate ORB similarity between two images
def calculate_orb_similarity(image_path1, image_path2):
    # Load the images
    image1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image_path2, cv2.IMREAD_GRAYSCALE)

    # Check if images were successfully loaded
    if image1 is None or image2 is None:
        print("Error loading one of the images.")
        return 0

    # Initiate ORB detector
    orb = cv2.ORB_create()

    # Find keypoints and descriptors with ORB
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

    # If no descriptors found in either image, return 0 matches
    if descriptors1 is None or descriptors2 is None:
        return 0

    # Use BFMatcher to find the matches
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Sort matches based on the distance between descriptors
    matches = sorted(matches, key=lambda x: x.distance)

    # Filter matches based on a distance threshold
    good_matches = [m for m in matches if m.distance < 30]

    # Return the ratio of good matches to the average number of keypoints
    return len(good_matches) / ((len(keypoints1) + len(keypoints2)) / 2)

# Function for the old version (if needed)
def calculate_orb_similarityold(image_path1, image_path2):
    # Load the images
    image1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image_path2, cv2.IMREAD_GRAYSCALE)

    # Check if images were successfully loaded
    if image1 is None or image2 is None:
        print("Error loading one of the images.")
        return 0

    # Initiate ORB detector
    orb = cv2.ORB_create()

    # Find keypoints and descriptors with ORB
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

    # Ensure descriptors are not None
    if descriptors1 is None or descriptors2 is None:
        print("No descriptors found in one or both images.")
        return 0

    # Use BFMatcher to find the matches
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Sort matches based on the distance between descriptors
    matches = sorted(matches, key=lambda x: x.distance)

    # Filter matches based on a distance threshold
    distance_threshold = 30
    good_matches = [m for m in matches if m.distance < distance_threshold]

    # Return the number of good matches
    return len(good_matches)




# Function to extract page number from the filename
def extract_page_number(filename):
    # Assuming the filename follows the pattern 'figure-page_number-something.jpg'
    parts = filename.split('-')
    if len(parts) > 1 and parts[1].isdigit():
        return int(parts[1])
    return None


# Extract the page number from the filename
df['Page_Number'] = df['Image_Path'].apply(lambda path: extract_page_number(os.path.basename(path)))

# Apply the function to calculate midpoints
df['Midpoint'] = df['Coordinates'].apply(calculate_midpoint)

# Add a column for the base filename
df['Image_Basename'] = df['Image_Path'].apply(lambda path: os.path.basename(path))

# Function to calculate the minimum distance between all coordinates (corners) of two images
def calculate_minimum_corner_distance(coords1, coords2):
    min_distance = float('inf')
    for c1 in coords1:
        for c2 in coords2:
            distance = calculate_distance(c1, c2)
            if distance < min_distance:
                min_distance = distance
    return min_distance

# Compare each pair of images, but only within the same page
pairwise_comparisons = []
grouped_by_page = df.groupby('Page_Number')


from skimage.metrics import structural_similarity as ssim
import numpy as np

# Function to calculate SSIM (Structural Similarity Index Measure)
def calculate_ssim(image_path1, image_path2):
    try:
        # Read the images
        image1 = cv2.imread(image_path1)
        image2 = cv2.imread(image_path2)
        
        # Check if images were successfully loaded
        if image1 is None or image2 is None:
            print("Error loading one of the images.")
            return None
        
        # Convert images to grayscale (if needed)
        if len(image1.shape) == 3:
            image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        if len(image2.shape) == 3:
            image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
        
        # Resize the second image to match the dimensions of the first image
        image2_resized = cv2.resize(image2, (image1.shape[1], image1.shape[0]))
        
        # Set win_size explicitly based on the smaller dimension of the images
        min_dimension = min(image1.shape[0], image1.shape[1])
        win_size = min(7, min_dimension) if min_dimension >= 7 else min_dimension
        
        # Ensure that win_size is odd
        if win_size % 2 == 0:
            win_size -= 1  # Make it odd by subtracting 1

        # Compute SSIM with the specified win_size
        score, _ = ssim(image1, image2_resized, win_size=win_size, full=True)
        return score
    
    except ZeroDivisionError:
        print("ZeroDivisionError encountered. Skipping this computation.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to calculate Canny edge detection similarity
def calculate_canny_similarity(image_path1, image_path2):
    # Read images
    image1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image_path2, cv2.IMREAD_GRAYSCALE)
    
    # Perform Canny edge detection
    edges1 = cv2.Canny(image1, 100, 200)
    edges2 = cv2.Canny(image2, 100, 200)
    
    # Resize if necessary
    if edges1.shape != edges2.shape:
        edges2 = cv2.resize(edges2, (edges1.shape[1], edges1.shape[0]))
    
    # Calculate the absolute difference between the two edge maps
    difference = cv2.absdiff(edges1, edges2)
    
    # Calculate the sum of differences (smaller values mean more similar)
    return np.sum(difference)

# Function to calculate histogram comparison
def calculate_histogram_similarity(image_path1, image_path2):
    # Read the images
    image1 = cv2.imread(image_path1)
    image2 = cv2.imread(image_path2)
    
    # Convert the images to HSV color space
    hsv_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2HSV)
    hsv_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
    # Calculate histograms
    hist1 = cv2.calcHist([hsv_image1], [0, 1], None, [50, 60], [0, 180, 0, 256])
    hist2 = cv2.calcHist([hsv_image2], [0, 1], None, [50, 60], [0, 180, 0, 256])
    
    # Normalize histograms
    cv2.normalize(hist1, hist1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    
    # Compare histograms using correlation method
    return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

# Function to calculate normalized histogram
def calculate_normalized_histogram(image_path, bins=256):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Calculate histogram
    hist = cv2.calcHist([image], [0], None, [bins], [0, 256])
    
    # Normalize the histogram
    hist = hist / np.sum(hist)
    
    return hist

def calculate_kl_divergence(image_path1, image_path2, bins=256):
    # Calculate normalized histograms
    hist1 = calculate_normalized_histogram(image_path1, bins)
    hist2 = calculate_normalized_histogram(image_path2, bins)
    
    # Calculate KL divergence
    kl_div = np.sum(rel_entr(hist1, hist2))  # KL divergence using scipy's relative entropy function
    return kl_div


# Function to calculate aspect ratio similarity
def calculate_aspect_ratio_similarity(image_path1, image_path2):
    size1 = get_image_size(image_path1)
    size2 = get_image_size(image_path2)
    
    aspect_ratio1 = size1[0] / size1[1]  # width / height
    aspect_ratio2 = size2[0] / size2[1]  # width / height
    
    # Return the difference in aspect ratio
    return abs(aspect_ratio1 - aspect_ratio2)

from scipy.spatial import distance
from scipy.stats import wasserstein_distance
from sklearn.metrics.pairwise import cosine_similarity

# Function to calculate Jensen-Shannon Divergence
def calculate_js_divergence(image_path1, image_path2, bins=256):
    # Calculate normalized histograms
    hist1 = calculate_normalized_histogram(image_path1, bins)
    hist2 = calculate_normalized_histogram(image_path2, bins)
    
    # Calculate Jensen-Shannon divergence
    js_div = distance.jensenshannon(hist1.flatten(), hist2.flatten())
    return js_div

# Function to calculate Earth Mover's Distance (EMD)

# def calculate_emd(image_path1, image_path2, bins=256):
#     # Read the images
#     image1 = cv2.imread(image_path1)
#     image2 = cv2.imread(image_path2)
    
#     # Check if images were successfully loaded
#     if image1 is None or image2 is None:
#         print("Error loading one of the images.")
#         return float('inf')

#     # Convert the images to HSV color space
#     hsv_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2HSV)
#     hsv_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
#     # Calculate histograms for the H and S channels
#     hist1 = cv2.calcHist([hsv_image1], [0, 1], None, [50, 60], [0, 180, 0, 256])
#     hist2 = cv2.calcHist([hsv_image2], [0, 1], None, [50, 60], [0, 180, 0, 256])
    
#     # Normalize histograms
#     cv2.normalize(hist1, hist1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
#     cv2.normalize(hist2, hist2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

#     # Reshape histograms into signature format for EMD
#     # Each row should be of the form [bin_value, frequency]
#     hist1 = hist1.reshape(-1, 1)
#     hist2 = hist2.reshape(-1, 1)
#     signature1 = np.zeros((hist1.shape[0], 2), dtype=np.float32)
#     signature2 = np.zeros((hist2.shape[0], 2), dtype=np.float32)
    
#     # Fill the signatures with index values and the corresponding histogram values
#     signature1[:, 0] = np.arange(hist1.shape[0])
#     signature1[:, 1] = hist1[:, 0]
#     signature2[:, 0] = np.arange(hist2.shape[0])
#     signature2[:, 1] = hist2[:, 0]

#     # Check if signatures contain non-zero values
#     if cv2.countNonZero(signature1[:, 1]) == 0 or cv2.countNonZero(signature2[:, 1]) == 0:
#         print("One of the signatures is empty (all zero). Returning a high EMD value.")
#         return float('inf')
    
#     # Compute Earth Mover's Distance
#     emd = cv2.EMD(signature1, signature2, cv2.DIST_L2)[0]
#     return emd

def calculate_emd(image_path1, image_path2):
    # Read the images
    image1 = cv2.imread(image_path1)
    image2 = cv2.imread(image_path2)
    
    # Check if images were successfully loaded
    if image1 is None or image2 is None:
        print("Error loading one of the images.")
        return None
    
    # Convert the images to HSV color space
    hsv_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2HSV)
    hsv_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    
    # Calculate histograms for the H and S channels
    hist1 = cv2.calcHist([hsv_image1], [0, 1], None, [50, 60], [0, 180, 0, 256])
    hist2 = cv2.calcHist([hsv_image2], [0, 1], None, [50, 60], [0, 180, 0, 256])
    
    # Normalize histograms
    cv2.normalize(hist1, hist1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    
    # Flatten histograms to 1D arrays
    hist1 = hist1.flatten().astype(np.float32)
    hist2 = hist2.flatten().astype(np.float32)
    
    # Check if either histogram is empty (contains all zeros)
    if np.count_nonzero(hist1) == 0 or np.count_nonzero(hist2) == 0:
        print("One of the histograms is empty (all zero bins). Cannot compute EMD.")
        return None
    
    # Create signature for the EMD calculation
    hist1_signature = np.column_stack((hist1, np.arange(len(hist1)).astype(np.float32)))
    hist2_signature = np.column_stack((hist2, np.arange(len(hist2)).astype(np.float32)))
    
    # Compute Earth Mover's Distance
    emd, _, _ = cv2.EMD(hist1_signature, hist2_signature, cv2.DIST_L2)
    return emd


# Function to calculate Bhattacharyya Distance
def calculate_bhattacharyya_distance(image_path1, image_path2, bins=256):
    # Calculate histograms
    hist1 = calculate_normalized_histogram(image_path1, bins)
    hist2 = calculate_normalized_histogram(image_path2, bins)
    
    # Calculate Bhattacharyya distance
    bhat_distance = cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)
    return bhat_distance

# Function to calculate Cosine Similarity between histograms
def calculate_cosine_similarity(image_path1, image_path2, bins=256):
    hist1 = calculate_normalized_histogram(image_path1, bins)
    hist2 = calculate_normalized_histogram(image_path2, bins)
    
    # Compute Cosine Similarity
    cos_sim = cosine_similarity(hist1.reshape(1, -1), hist2.reshape(1, -1))[0][0]
    return cos_sim

# Function to calculate Chi-Square Distance
def calculate_chi_square_distance(image_path1, image_path2, bins=256):
    # Calculate histograms
    hist1 = calculate_normalized_histogram(image_path1, bins)
    hist2 = calculate_normalized_histogram(image_path2, bins)
    
    # Compute Chi-Square distance
    chi_square = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CHISQR)
    return chi_square

# Function to perform template matching with height truncation
def calculate_template_match_percentage(image_path1, image_path2):
    # Load the images
    image1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image_path2, cv2.IMREAD_GRAYSCALE)
    
    h1, w1 = image1.shape
    h2, w2 = image2.shape
    
    # Truncate the height of the larger image to the height of the smaller image
    
    try:
        # Perform template matching
        result = cv2.matchTemplate(image2, image1, cv2.TM_SQDIFF_NORMED)
        # Get the best match position (max value and location)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        match_percentage = max_val * 100
    except :
        try :
        # Perform template matching
            result = cv2.matchTemplate(image1, image2, cv2.TM_SQDIFF_NORMED)
            # Get the best match position (max value and location)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            match_percentage = max_val * 100
        except :
            if h1 < h2:
                image2 = image2[:h1, :]  # Truncate image2 to match the height of image1
            elif h2 < h1:
                image1 = image1[:h2, :]  # Truncate image1 to match the height of image2
            try :
                result = cv2.matchTemplate(image2, image1, cv2.TM_SQDIFF_NORMED)
                # Get the best match position (max value and location)
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
                match_percentage = max_val * 100
            except : 
                try :
                    result = cv2.matchTemplate(image1, image2, cv2.TM_SQDIFF_NORMED)
                    # Get the best match position (max value and location)
                    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
                    match_percentage = max_val * 100
                except :
                    print(f"Error during template matching: {e}")
                    return -100  # Return 0% if any error occurs
    return match_percentage

# Iterate over each page group and compare images on the same page
for page_number, group in grouped_by_page:
    for (i, row1), (j, row2) in itertools.combinations(group.iterrows(), 2):
        # Calculate midpoint distance
        midpoint_distance = calculate_distance(row1['Midpoint'], row2['Midpoint'])
        
        # Calculate hash difference
        hash_difference = calculate_hash_difference(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate ORB similarity
        orb_similarity = calculate_orb_similarityold(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate ORB similarity
        orb_similarity_perc = calculate_orb_similarity(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate SSIM
        ssim_value = calculate_ssim(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate Canny edge similarity
        canny_similarity = calculate_canny_similarity(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate histogram similarity
        histogram_similarity = calculate_histogram_similarity(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate aspect ratio similarity
        aspect_ratio_similarity = calculate_aspect_ratio_similarity(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate KL divergence
        kl_divergence = calculate_kl_divergence(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate Jensen-Shannon Divergence (JSD)
        js_divergence = calculate_js_divergence(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate Earth Mover's Distance (EMD)
        emd_value = calculate_emd(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate Bhattacharyya Distance
        bhattacharyya_distance = calculate_bhattacharyya_distance(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate Cosine Similarity
        cosine_similarity_value = calculate_cosine_similarity(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate Chi-Square Distance
        chi_square_distance = calculate_chi_square_distance(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate template matching percentage
        template_match_percentage = calculate_template_match_percentage(row1['Image_Path'], row2['Image_Path'])
        
        # Calculate image sizes
        size1 = get_image_size(row1['Image_Path'])
        size2 = get_image_size(row2['Image_Path'])
        
        # Calculate areas
        area1 = size1[0] * size1[1]
        area2 = size2[0] * size2[1]
        
        # Calculate minimum corner distance
        min_corner_distance = calculate_minimum_corner_distance(row1['Coordinates'], row2['Coordinates'])
        
        # Store the comparison results
        pairwise_comparisons.append({
            'Page_Number': page_number,
            'Image1': row1['Image_Basename'],
            'Image2': row2['Image_Basename'],
            'Midpoint_Distance': midpoint_distance,
            'Hash_Difference': hash_difference,
            'ORB_Similarity': orb_similarity,
            'ORB_Similarity_Perc': orb_similarity_perc,
            'SSIM': ssim_value,
            'Canny_Similarity': canny_similarity,
            'Histogram_Similarity': histogram_similarity,
            'Aspect_Ratio_Similarity': aspect_ratio_similarity,
            'KL_Divergence': kl_divergence,
            'JS_Divergence': js_divergence,  # Added JS divergence result
            'EMD': emd_value,  # Added EMD result
            'Bhattacharyya_Distance': bhattacharyya_distance,  # Added Bhattacharyya result
            'Cosine_Similarity': cosine_similarity_value,  # Added Cosine Similarity result
            'Chi_Square_Distance': chi_square_distance,  # Added Chi-Square Distance result
            'Template_Match_Percentage': template_match_percentage,  # Added template matching result
            'Area_Image1': area1,
            'Area_Image2': area2,
            'Min_Corner_Distance': min_corner_distance
        })

# Convert the results into a DataFrame for easy analysis
comparison_df = pd.DataFrame(pairwise_comparisons)

# Display the comparison DataFrame
print(comparison_df)

No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or both images.
No descriptors found in one or b

KeyboardInterrupt: 

In [ ]:
#not to use in paper guide 

to_remove_size_1 = comparison_df[(comparison_df['Area_Image1']<100000)]
to_remove_size_2 = comparison_df[(comparison_df['Area_Image2']<100000)]
to_remove_size_list = to_remove_size_1['Image1'].append(to_remove_size_2['Image2']).drop_duplicates()
comparison_df = comparison_df[(comparison_df['Area_Image2']>100000)& (comparison_df['Area_Image1']>100000)]

import numpy as np

# Apply conditional logic for the area proportion
comparison_df['area_prop'] = np.where(comparison_df['Area_Image1'] < comparison_df['Area_Image2'],
                                      comparison_df['Area_Image1'] / comparison_df['Area_Image2'],
                                      comparison_df['Area_Image2'] / comparison_df['Area_Image1'])

to_check_further = comparison_df[((((comparison_df['Min_Corner_Distance'] <20) & (comparison_df['ORB_Similarity'] >5)) | (comparison_df['ORB_Similarity'] >15)) &  (comparison_df['Template_Match_Percentage'] >5))]

#to_check_further = to_check_further[(to_check_further['ORB_Similarity_Perc']>0.135) & (to_check_further['area_prop']<0.65) ].sort_values("area_prop") #for liver imaging book


to_check_further = to_check_further[(to_check_further['ORB_Similarity_Perc']>0.135)].sort_values("area_prop") #for mayo book


to_check_further['Smaller_Image'] = to_check_further.apply(
    lambda row: row['Image1'] if row['Area_Image1'] < row['Area_Image2'] else (
        row['Image2'] if row['Area_Image2'] < row['Area_Image1'] else 'Same Size'),
    axis=1
)

to_remove = pd.DataFrame(to_check_further['Smaller_Image'].drop_duplicates()).reset_index(drop = True)

clean_df = df[~df['Image_Basename'].isin(to_remove['Smaller_Image'])]

#clean_df = df[~df['Image_Basename'].isin(["figure-67-84.jpg","figure-66-81.jpg"])] #only for mayo
clean_df = clean_df[~clean_df['Image_Basename'].isin(to_remove_size_list)]


In [382]:
df

,Element_Type,Text,Page_Number,Image_Path,Coordinates,coordinate_bottom_left,Midpoint,Image_Basename
0,image,,1,./Diagnostic Imaging_ Gastrointestinal/figure-...,"((-25.183333333333334, -29.43333333333334), (-...","(-25.183333333333334, 514.1)","(324.41666666666663, 242.33333333333331)",figure-1-1.jpg
1,image,,1,./Diagnostic Imaging_ Gastrointestinal/figure-...,"((669.65, -25.10000000000016), (669.65, 1114.4...","(669.65, 1114.4333333333334)","(1198.75, 544.6666666666665)",figure-1-2.jpg
2,image,,1,./Diagnostic Imaging_ Gastrointestinal/figure-...,"((-28.849999999999998, 504.7333333333332), (-2...","(-28.849999999999998, 1123.1)","(343.58333333333326, 813.9166666666665)",figure-1-3.jpg
3,image,,3,./Diagnostic Imaging_ Gastrointestinal/figure-...,"((-1.6990833333333335, 352.39722222222235), (-...","(-1.6990833333333335, 579.4194444444445)","(765.4536944444444, 465.9083333333334)",figure-3-4.jpg
4,image,,3,./Diagnostic Imaging_ Gastrointestinal/figure-...,"((1529.8166666666666, 352.39722222222235), (15...","(1529.8166666666666, 579.4194444444445)","(1616.3791666666666, 465.9083333333334)",figure-3-5.jpg
...,...,...,...,...,...,...,...,...
6376,image,,1071,./Diagnostic Imaging_ Gastrointestinal/figure-...,"((682.4833333333332, 1465.8999999999999), (682...","(682.4833333333332, 1473.4333333333332)","(933.25, 1469.6666666666665)",figure-1071-516.jpg
6377,image,,1071,./Diagnostic Imaging_ Gastrointestinal/figure-...,"((149.31666666666666, 1959.8999999999999), (14...","(149.31666666666666, 1967.4333333333332)","(400.0, 1963.6666666666665)",figure-1071-517.jpg
6378,image,,1071,./Diagnostic Imaging_ Gastrointestinal/figure-...,"((682.4833333333332, 1959.8999999999999), (682...","(682.4833333333332, 1967.4333333333332)","(933.25, 1963.6666666666665)",figure-1071-518.jpg
6379,image,,1073,./Diagnostic Imaging_ Gastrointestinal/figure-...,"((-25.016666666666666, 208.23333333333318), (-...","(-25.016666666666666, 475.09999999999985)","(850.3333333333333, 341.6666666666665)",figure-1073-519.jpg


In [383]:
clean_df = df

In [384]:
import pickle
with open(pickle_file_path, 'wb') as f:
    pickle.dump([image_elements, captions, raw_elements, df_flattened, clean_df], f)

20241022 - delete all 'duplicated' images

In [ ]:
import os
import shutil  # To move files from tmp to the final folder
from typing import Any
from pydantic import BaseModel
import time
from PyPDF2 import PdfFileReader, PdfFileWriter
from unstructured.partition.pdf import partition_pdf

class Element(BaseModel):
    type: str
    text: Any
    page_number: int
    table_as_html: Any
    image_path: str
    coordinate : Any

In [244]:
import pickle

# Define the path to your pickle file
pickle_file_path = "/home/jupyter/project/VP_storage/output_books/gastrointestinal-imaging-the-requisites-fourth-edition.pkl"

In [245]:
with open(pickle_file_path, 'rb') as file:
    image_elements, captions, raw_elements, df_flattened, clean_df= pickle.load(file)

In [246]:
clean_df

,Element_Type,Text,Page_Number,Image_Path,Coordinates,Midpoint,Image_Basename,coordinate_bottom_left
0,image,A-ring Esophageal vestibule Z line B-ring ...,1,./gastrointestinal-imaging-the-requisites-four...,"((928.35009765625, 462.1430358886719), (928.35...","(1238.4750488281252, 773.4558563232422)",figure-1-1.jpg,"(928.35009765625, 1084.7686767578125)"
1,image,B C D E,2,./gastrointestinal-imaging-the-requisites-four...,"((329.6348571777344, 1575.3785400390625), (329...","(815.5081634521484, 1783.7391357421875)",figure-2-2.jpg,"(329.6348571777344, 1992.0997314453125)"
2,image,,3,./gastrointestinal-imaging-the-requisites-four...,"((224.96660027777776, 1307.955508888889), (224...","(508.33326986111103, 1661.9888519444444)",figure-3-3.jpg,"(224.96660027777776, 2016.0221949999998)"
3,image,,3,./gastrointestinal-imaging-the-requisites-four...,"((992.966461111111, 1191.6332669444446), (992....","(1241.666497083333, 1591.6666243055556)",figure-3-4.jpg,"(992.966461111111, 1991.6999816666666)"
5,image,A B,4,./gastrointestinal-imaging-the-requisites-four...,"((297.73480224609375, 190.07211303710938), (29...","(837.7347106933594, 577.8328094482422)",figure-4-6.jpg,"(297.73480224609375, 965.593505859375)"
...,...,...,...,...,...,...,...,...
1288,image,,391,./gastrointestinal-imaging-the-requisites-four...,"((541.6332669444445, 166.63254638888878), (541...","(824.999978888889, 384.3325933333332)",figure-391-306.jpg,"(541.6332669444445, 602.0326402777777)"
1289,image,,391,./gastrointestinal-imaging-the-requisites-four...,"((541.6666666666666, 740.6053333333334), (541....","(825.0, 990.9386530555556)",figure-391-307.jpg,"(541.6666666666666, 1241.2719727777778)"
1290,image,,391,./gastrointestinal-imaging-the-requisites-four...,"((541.6332669444445, 1379.8449197222221), (541...","(824.9998516666666, 1582.878303472222)",figure-391-308.jpg,"(541.6332669444445, 1785.911687222222)"
1292,image,A C B D,392,./gastrointestinal-imaging-the-requisites-four...,"((296.62335205078125, 169.48562622070312), (29...","(876.1047668457031, 626.5541534423828)",figure-392-310.jpg,"(296.62335205078125, 1083.6226806640625)"


In [385]:
main_directory = "/home/jupyter/project/VP_storage/output_books/"

In [386]:
import os
import pickle
import pandas as pd

# Initialize accumulators for each element in the .pkl files
all_image_elements = []
all_captions = []
all_raw_elements = []
all_df_flattened = []
all_clean_df = []

# Get all subfolders in the main directory
subfolders = [subfolder for subfolder in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subfolder))]
subfolders = [subfolder for subfolder in subfolders if subfolder != ".ipynb_checkpoints"]
subfolders = [subfolder for subfolder in subfolders if subfolder != "gastrointestinal-imaging-the-requisites-fourth-edition_11pager.pdf"]

for subfolder in subfolders:
    subfolder_path = os.path.join(main_directory, subfolder)
    if os.path.isdir(subfolder_path):
        # Construct the path for the corresponding .pkl file
        pkl_file_path = os.path.join(main_directory, f"{subfolder}.pkl")
        
        # Load the clean_df from the .pkl file if it exists
        if os.path.exists(pkl_file_path):
            print(pkl_file_path)
            with open(pkl_file_path, 'rb') as file:
                image_elements, captions, raw_elements, df_flattened, clean_df = pickle.load(file)
                
                # Append each loaded element to its respective accumulator
                all_image_elements.append(image_elements)
                all_captions.append(captions)
                all_raw_elements.append(raw_elements)
                all_df_flattened.append(df_flattened)
                all_clean_df.append(clean_df)
            
            # Extract only the basenames from the clean_df
            basenames_to_keep = clean_df['Image_Path'].apply(lambda x: os.path.basename(x)).values
            
            # Iterate through files in the subfolder and delete the ones not in the basenames_to_keep
            for filename in os.listdir(subfolder_path):
                if filename not in basenames_to_keep and filename != ".ipynb_checkpoints"   :
                    # If the file's basename is not in clean_df, delete it
                    file_path = os.path.join(subfolder_path, filename)
                    os.remove(file_path)

# Combine all DataFrames in all_clean_df into a single DataFrame, if needed
combined_clean_df = pd.concat(all_clean_df, ignore_index=True)

# Checking the remaining files in each subfolder to verify the process after modification
remaining_files_per_subfolder = {
    subfolder: os.listdir(os.path.join(main_directory, subfolder))
    for subfolder in subfolders
}

# Optional: display the combined DataFrame or the lists to verify accumulation
print(combined_clean_df.head())
print(remaining_files_per_subfolder)

/home/jupyter/project/VP_storage/output_books/Diagnostic Imaging_ Gastrointestinal.pkl
/home/jupyter/project/VP_storage/output_books/CT and MRI of the Whole Body, 2-Volume Set, 6e, Volume I.pkl
/home/jupyter/project/VP_storage/output_books/Liver imaging _ MRI with CT correlation.pkl
/home/jupyter/project/VP_storage/output_books/Radiology Illustrated_ Hepatobiliary and Pancreatic Radiology.pkl
/home/jupyter/project/VP_storage/output_books/Mayo Clinic Gastrointestinal Imaging Review.pkl
/home/jupyter/project/VP_storage/output_books/gastrointestinal-imaging-the-requisites-fourth-edition.pkl
  Element_Type Text  Page_Number  \
0        image                 1   
1        image                 1   
2        image                 1   
3        image                 3   
4        image                 3   

                                          Image_Path  \
0  ./Diagnostic Imaging_ Gastrointestinal/figure-...   
1  ./Diagnostic Imaging_ Gastrointestinal/figure-...   
2  ./Diagnostic Imag

In [388]:
import pickle
with open("/home/jupyter/project/VP_storage/output_books_all.pkl", 'wb') as f:
    pickle.dump([all_image_elements, all_captions, all_raw_elements, all_df_flattened, combined_clean_df], f)

In [389]:
main_directory = "/home/jupyter/project/VP_storage/output_papers/"

In [390]:
import os
import pickle
import pandas as pd

# Initialize accumulators for each element in the .pkl files
all_image_elements = []
all_captions = []
all_raw_elements = []
all_df_flattened = []
all_clean_df = []

# Get all subfolders in the main directory
subfolders = [subfolder for subfolder in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subfolder))]
subfolders = [subfolder for subfolder in subfolders if subfolder != ".ipynb_checkpoints"]

for subfolder in subfolders:
    subfolder_path = os.path.join(main_directory, subfolder)
    if os.path.isdir(subfolder_path):
        # Construct the path for the corresponding .pkl file
        pkl_file_path = os.path.join(main_directory, f"{subfolder}.pkl")
        
        # Load the clean_df from the .pkl file if it exists
        if os.path.exists(pkl_file_path):
            with open(pkl_file_path, 'rb') as file:
                image_elements, captions, raw_elements, df_flattened, clean_df = pickle.load(file)
                
                # Append each loaded element to its respective accumulator
                all_image_elements.append(image_elements)
                all_captions.append(captions)
                all_raw_elements.append(raw_elements)
                all_df_flattened.append(df_flattened)
                all_clean_df.append(clean_df)
            
            # Extract only the basenames from the clean_df
            basenames_to_keep = clean_df['Image_Path'].apply(lambda x: os.path.basename(x)).values
            
            # Iterate through files in the subfolder and delete the ones not in the basenames_to_keep
            for filename in os.listdir(subfolder_path):
                if filename not in basenames_to_keep and filename != ".ipynb_checkpoints":
                    # If the file's basename is not in clean_df, delete it
                    file_path = os.path.join(subfolder_path, filename)
                    os.remove(file_path)

# Combine all DataFrames in all_clean_df into a single DataFrame, if needed
combined_clean_df = pd.concat(all_clean_df, ignore_index=True)

# Checking the remaining files in each subfolder to verify the process after modification
remaining_files_per_subfolder = {
    subfolder: os.listdir(os.path.join(main_directory, subfolder))
    for subfolder in subfolders
}

# Optional: display the combined DataFrame or the lists to verify accumulation
print(combined_clean_df.head())
print(remaining_files_per_subfolder)

  Element_Type                                               Text  \
0        image  a ie Meee Cle Arterial phase Portal venous pha...   
1        image             Ne F Portal venous phase Delayed phase   
2        image  Figure  3.  Sample  structured  re-  ports  wi...   
3        image                          ~ al eit outed urs areal)   
4        image  Treated observation degradation or omission Ot...   

   Page_Number                                         Image_Path  \
0            5  ./m-cunha-et-al-2021-how-to-use-li-rads-to-rep...   
1            5  ./m-cunha-et-al-2021-how-to-use-li-rads-to-rep...   
2            7  ./m-cunha-et-al-2021-how-to-use-li-rads-to-rep...   
3            8  ./m-cunha-et-al-2021-how-to-use-li-rads-to-rep...   
4            9  ./m-cunha-et-al-2021-how-to-use-li-rads-to-rep...   

                                         Coordinates  \
0  ((183.33333333333331, 140.9998611111108), (183...   
1  ((183.33333333333331, 547.9444722222221), (183...   
2 

In [391]:
import pickle
with open("/home/jupyter/project/VP_storage/output_papers_all.pkl", 'wb') as f:
    pickle.dump([all_image_elements, all_captions, all_raw_elements, all_df_flattened, combined_clean_df], f)

In [393]:
main_directory = "/home/jupyter/project/VP_storage/output_system_guides/"

In [394]:
import os
import pickle
import pandas as pd

# Initialize accumulators for each element in the .pkl files
all_image_elements = []
all_captions = []
all_raw_elements = []
all_df_flattened = []
all_clean_df = []

# Get all subfolders in the main directory
subfolders = [subfolder for subfolder in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subfolder))]
subfolders = [subfolder for subfolder in subfolders if subfolder != ".ipynb_checkpoints"]

for subfolder in subfolders:
    subfolder_path = os.path.join(main_directory, subfolder)
    if os.path.isdir(subfolder_path):
        # Construct the path for the corresponding .pkl file
        pkl_file_path = os.path.join(main_directory, f"{subfolder}.pkl")
        
        # Load the clean_df from the .pkl file if it exists
        if os.path.exists(pkl_file_path):
            with open(pkl_file_path, 'rb') as file:
                image_elements, captions, raw_elements, df_flattened, clean_df = pickle.load(file)
                
                # Append each loaded element to its respective accumulator
                all_image_elements.append(image_elements)
                all_captions.append(captions)
                all_raw_elements.append(raw_elements)
                all_df_flattened.append(df_flattened)
                all_clean_df.append(clean_df)
            
            # Extract only the basenames from the clean_df
            basenames_to_keep = clean_df['Image_Path'].apply(lambda x: os.path.basename(x)).values
            
            # Iterate through files in the subfolder and delete the ones not in the basenames_to_keep
            for filename in os.listdir(subfolder_path):
                if filename not in basenames_to_keep and filename != ".ipynb_checkpoints":
                    # If the file's basename is not in clean_df, delete it
                    file_path = os.path.join(subfolder_path, filename)
                    os.remove(file_path)

# Combine all DataFrames in all_clean_df into a single DataFrame, if needed
combined_clean_df = pd.concat(all_clean_df, ignore_index=True)

# Checking the remaining files in each subfolder to verify the process after modification
remaining_files_per_subfolder = {
    subfolder: os.listdir(os.path.join(main_directory, subfolder))
    for subfolder in subfolders
}

# Optional: display the combined DataFrame or the lists to verify accumulation
print(combined_clean_df.head())
print(remaining_files_per_subfolder)

  Element_Type                                               Text  \
0        image                     American College of Radiology™   
1        image                                                      
2        image                LR-TR Nonprogressing  LR-TR Viable    
3        image  LR-TR Nonviable  LR-TR Nonprogressing  LR-TR V...   
4        image                                                 •    

   Page_Number                                         Image_Path  \
0            1  ./LI-RADS CTMR Radiation TRA v2024 Core/figure...   
1            1  ./LI-RADS CTMR Radiation TRA v2024 Core/figure...   
2            2  ./LI-RADS CTMR Radiation TRA v2024 Core/figure...   
3            2  ./LI-RADS CTMR Radiation TRA v2024 Core/figure...   
4            2  ./LI-RADS CTMR Radiation TRA v2024 Core/figure...   

                                         Coordinates  \
0  ((31.0, 19.666666666666778), (31.0, 254.000000...   
1  ((489.66666666666663, 926.4999999999999), (489...   
2 

In [395]:
combined_clean_df

,Element_Type,Text,Page_Number,Image_Path,Coordinates,coordinate_bottom_left,Midpoint,Image_Basename
0,image,American College of Radiology™,1,./LI-RADS CTMR Radiation TRA v2024 Core/figure...,"((31.0, 19.666666666666778), (31.0, 254.000000...","(31.0, 254.00000000000014)","(163.16666666666666, 136.83333333333346)",figure-1-1.jpg
1,image,,1,./LI-RADS CTMR Radiation TRA v2024 Core/figure...,"((489.66666666666663, 926.4999999999999), (489...","(489.66666666666663, 1447.3333333333333)","(750.0833333333333, 1186.9166666666665)",figure-1-2.jpg
2,image,LR-TR Nonprogressing LR-TR Viable,2,./LI-RADS CTMR Radiation TRA v2024 Core/figure...,"((303.5446472167969, 1058.6531982421875), (303...","(303.5446472167969, 1171.836181640625)","(728.4785614013672, 1115.2446899414062)",figure-2-3.jpg
3,image,LR-TR Nonviable LR-TR Nonprogressing LR-TR V...,2,./LI-RADS CTMR Radiation TRA v2024 Core/figure...,"((169.9696044921875, 1307.6951904296875), (169...","(169.9696044921875, 1382.4962158203125)","(709.7643432617188, 1345.095703125)",figure-2-4.jpg
4,image,•,2,./LI-RADS CTMR Radiation TRA v2024 Core/figure...,"((5.166666666666667, 1581.8333333333335), (5.1...","(5.166666666666667, 1721.75)","(90.29166666666666, 1651.791666666667)",figure-2-5.jpg
...,...,...,...,...,...,...,...,...
142,image,LI-RADS LI-RADS _ Prior Exam _ Exam Current Ex...,3,./What's New in v2018 _ RADS - Reporting and D...,"((166.66665972222222, 704.166637326389), (166....","(166.66665972222222, 1454.166606076389)","(886.4582963975693, 1079.166621701389)",figure-3-1.jpg
143,image,LI-RADS LI-RADS v2017 v2018 Visible on surveil...,5,./What's New in v2018 _ RADS - Reporting and D...,"((93.74999609375, 181.24999244791664), (93.749...","(93.74999609375, 1952.0832519965277)","(849.9999645833332, 1066.6666222222223)",figure-5-2.jpg
144,image,Size Nonrim — Nonperipheral 10-19 mm APHE “was...,6,./What's New in v2018 _ RADS - Reporting and D...,"((166.66665972222222, 808.3332996527777), (166...","(166.66665972222222, 1104.1666206597222)","(886.4582963975693, 956.2499601562499)",figure-6-3.jpg
145,image,Features LI-RADS v2018 OPTN 10-19 mm + APHE + ...,7,./What's New in v2018 _ RADS - Reporting and D...,"((166.66665972222222, 129.1666612847223), (166...","(166.66665972222222, 1016.6666243055555)","(886.4582963975693, 572.9166427951388)",figure-7-4.jpg


In [397]:
import pickle
with open("/home/jupyter/project/VP_storage/output_system_guides_all.pkl", 'wb') as f:
    pickle.dump([all_image_elements, all_captions, all_raw_elements, all_df_flattened, combined_clean_df], f)